In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
import pickle
import os
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
import sklearn
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import pickle

In [4]:
def load_data():
    column_labels = ["time", "x", "y", "z", "activity"]
    data = pd.DataFrame()
    temp=[]
    for i in range(1,16):
        df = pd.read_csv(str(i) + ".csv", names = column_labels)
        temp.append(df)
    data = pd.concat(temp)
    return data

def normalize(col):
    mean = np.mean(col,axis = 0)
    sig = np.std(col,axis = 0)
    return (col - mean)/sig

def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)
        
def segment_signal(data,window_size = 200):
    segments = np.empty((0,window_size,3))
    labels = np.empty((0))
    for (start, end) in windows(data["time"], window_size):
        start = int(start)
        end = int(end)
        x = data["x"][start:end]
        y = data["y"][start:end]
        z = data["z"][start:end]
        if(len(data["time"][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([x,y,z])])
            labels = np.append(labels,stats.mode(data["activity"][start:end])[0][0])
    return segments, labels

def get_batches(X, y, batch_size = 100):
    """ Return a generator for batches """
    n_batches = len(X) // batch_size
    X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

    # Loop over batches and yield
    for b in range(0, len(X), batch_size):
        yield X[b:b+batch_size], y[b:b+batch_size]

In [5]:
data = load_data()
data = data[data.activity != 0]
data['x'] = normalize(data['x'])
data['y'] = normalize(data['y'])
data['z'] = normalize(data['z'])

In [6]:
# segments, labels = segment_signal(data)
# labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

In [7]:
# pickle.dump( segments, open( "segments.p", "wb" ) )
# pickle.dump( labels, open( "labels.p", "wb" ) )

In [8]:
segments = pickle.load( open( "segments.p", "rb" ) )
labels = pickle.load( open( "labels.p", "rb" ) )

In [9]:
split = np.random.rand(len(segments)) < 0.70
X_train = segments[split]
y_train = labels[split]
X_test = segments[~split]
y_test = labels[~split]

In [10]:
# print (X_train.shape , y_train.shape)
# print (X_test.shape , y_test.shape)

In [11]:
lstm_size = 36         # 2 times the amount of channels
lstm_layers = 2        # Number of layers
batch_size = 600       # Batch size
seq_len = 200          # Number of steps
learning_rate = 0.0005  # Learning rate (default is 0.001)
epochs = 200

# Fixed
n_classes = 7
n_channels = 3

In [12]:
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

In [29]:
conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=2, strides=1, 
                         padding='same', activation = tf.nn.relu)
n_ch = n_channels *2

lstm_input = tf.transpose(conv1, [1,0,2])
lstm_input = tf.reshape(lstm_input, [-1, n_ch])

lstm_input = tf.layers.dense(lstm_input, lstm_size, activation=None)
lstm_input = tf.split(lstm_input, seq_len, 0)

lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
dropout = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_)
cell = tf.contrib.rnn.MultiRNNCell([dropout] * lstm_layers)
initial_state = cell.zero_state(batch_size, tf.float32)

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_3/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(600, 36) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_3/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(600, 36) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_3/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros:0' shape=(600, 36) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_3/DropoutWrapperZeroState_1/BasicLSTMCellZeroState/zeros_1:0' shape=(600, 36) dtype=float32>))

In [19]:
outputs, final_state = tf.contrib.rnn.static_rnn(cell, lstm_input, dtype=tf.float32,
                                                 initial_state = initial_state)
logits = tf.layers.dense(outputs[-1], n_classes, name='logits')
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)

#     train_op = tf.train.AdamOptimizer(learning_rate_)
#     gradients = train_op.compute_gradients(cost)
#     capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
#     optimizer = train_op.apply_gradients(capped_gradients)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [20]:
# if (os.path.exists('crnn-saved') == False):
#     !mkdir checkpoints-crnn

A subdirectory or file checkpoints-crnn already exists.


In [21]:
test_acc = []
test_loss = []

train_acc = []
train_loss = []

# with graph.as_default():
saver = tf.train.Saver()

# with tf.Session(graph=graph) as sess:
# with tf.Session() as sess:

sess = tf.Session()
sess.run(tf.global_variables_initializer())
# writer = tf.summary.FileWriter("logs/", sess.graph)

iteration = 1

for e in range(epochs):
#     tf.set_random_seed(123)
    # Loop over batches
    
    state = sess.run(initial_state)
    for x,y in get_batches(X_train, y_train, batch_size):

        # Feed dictionary
        feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, 
                initial_state : state, learning_rate_ : learning_rate}

        # Loss
        loss, _ , state, acc = sess.run([cost, optimizer, final_state, accuracy], 
                                             feed_dict = feed)
        train_acc.append(acc)
        train_loss.append(loss)

        # Print at each 5 iters
        if (iteration % 5 == 0):
            print("Epoch: {}/{}".format(e, epochs),
                  "Iteration: {:d}".format(iteration),
                  "Train loss: {:6f}".format(loss),
                  "Train acc: {:.6f}".format(acc))

        # Compute validation loss at every 10 iterations
        if (iteration%10 == 0):
            val_state = sess.run(cell.zero_state(batch_size, tf.float32))
            val_acc_ = []
            val_loss_ = []

            for x_t, y_t in get_batches(X_test, y_test, batch_size):
                # Feed
                feed = {inputs_ : x_t, labels_ : y_t, 
                        keep_prob_ : 1.0, initial_state : val_state}  

                # Loss
                loss_v, state_v, acc_v = sess.run([cost, final_state,accuracy], feed_dict = feed)                    
                val_acc_.append(acc_v)
                val_loss_.append(loss_v)

            # Print info
            print("Epoch: {}/{}".format(e, epochs),
                  "Iteration: {:d}".format(iteration),
                  "Testing loss NOW: {:6f}".format(np.mean(val_loss_)),
                  "Testing acc NOW: {:.6f}".format(np.mean(val_acc_)))

            # Store
            test_acc.append(np.mean(val_acc_))
            test_loss.append(np.mean(val_loss_))

        # Iterate 
        iteration += 1

    print("Optimization Finished!")
print("Ended!")
        
saver.save(sess,"crnn-saved/test.ckpt")

InvalidArgumentError: All dimensions except 1 must match. Input 1 has shape [600 36] and doesn't match input 0 with shape [1800 36].
	 [[Node: gradients/rnn/rnn/multi_rnn_cell/cell_0/cell_0/basic_lstm_cell/basic_lstm_cell_400/concat_grad/ConcatOffset = ConcatOffset[N=2, _device="/job:localhost/replica:0/task:0/cpu:0"](gradients/rnn/rnn/multi_rnn_cell/cell_0/cell_0/basic_lstm_cell/basic_lstm_cell_400/concat_grad/mod, gradients/rnn/rnn/multi_rnn_cell/cell_0/cell_0/basic_lstm_cell/basic_lstm_cell_400/concat_grad/ShapeN, gradients/rnn/rnn/multi_rnn_cell/cell_0/cell_0/basic_lstm_cell/basic_lstm_cell_400/concat_grad/ShapeN:1)]]

Caused by op 'gradients/rnn/rnn/multi_rnn_cell/cell_0/cell_0/basic_lstm_cell/basic_lstm_cell_400/concat_grad/ConcatOffset', defined at:
  File "C:\Users\Wayne\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Wayne\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globa